In [24]:
%pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [25]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import shutil
import requests
import csv
import os
import pickle

In [34]:
class StudentPortalSite_NoticeLine():
    def __init__(self,USERNAME,PASSWORD):
        self.USERNAME = USERNAME
        self.PASSWORD = PASSWORD
        


        ##ポータルサイト
        Target_URL = "https://ptl.bunri-u.ac.jp/BunriPortal/studentMenu.jsf"
        service = Service(executable_path=r'C:\Users\gahyi\python勉強\VScode\ポートフォリオ\chromedriver-win64\chromedriver.exe')
        options = webdriver.ChromeOptions()

        options.add_argument('--headless')
        driver = webdriver.Chrome(service=service,options=options)
        driver.get(Target_URL)
        sleep(1)

        ##ポータルサイトの自動ログイン
        USERNAME_input = driver.find_element(By.XPATH,"//input[@name='jp.ac.bunri.login.USERNAME']")
        USERNAME_input.send_keys(self.USERNAME)
        sleep(1)

        PASSWORD_input = driver.find_element(By.XPATH,"//input[@name='jp.ac.bunri.login.PASSWORD']")
        PASSWORD_input.send_keys(self.PASSWORD)
        sleep(1)

        LOGIN_input = driver.find_element(By.XPATH,("//input[@type='submit']"))
        LOGIN_input.submit()
        sleep(1)



        ##csvファイル名を定義
        new_csv_CallInfo = self.USERNAME + "個人呼び出しnew.csv"
        new_csv_NoticeLesson = self.USERNAME + "授業のお知らせnew.csv"
        # new_csv_NoticeLesson = USERNAME + "授業の情報new.csv"

        ##csvファイルをopen
        #個人呼び出し
        csv_CallInfo_dir = open(new_csv_CallInfo,"w",encoding="cp932")
        writer_NewCallInfo = csv.writer(csv_CallInfo_dir,lineterminator="\n")

        #授業のお知らせ
        csv_NoticeLesson_dir = open(new_csv_NoticeLesson,"w",encoding="cp932")
        writer_NewNoticeLesson = csv.writer(csv_NoticeLesson_dir,lineterminator="\n")

        #授業の情報
        # csv_NoticeLesson_dir = open(new_csv_NoticeLesson,"w",encoding="cp932")
        # writer_NewNoticeLesson = csv.writer(csv_NoticeLesson_dir,lineterminator="\n")



        ##HTMLのテキストをcsvファイルに保存
        #個人呼び出し
        try:
            for elem_tr_CallInfo in driver.find_elements(By.XPATH,'//*[@id="callInfoPublicListForm:_id2:tbody_element"]/tr')[0::2]:
                csvlist_CallInfo = []
                csvlist_CallInfo.append(elem_tr_CallInfo.text)
                writer_NewCallInfo.writerow(csvlist_CallInfo)
        except:
            print("個人呼び出しの指定のidが存在しない")

        #授業のお知らせ
        try:
            for elem_tr_NoticeLesson in driver.find_elements(By.XPATH,'//*[@id="noticeLessonPublicListForm:_id26:tbody_element"]/tr')[0::2]:
                csvlist_NoticeLesson = []
                csvlist_NoticeLesson.append(elem_tr_NoticeLesson.text)
                writer_NewNoticeLesson.writerow(csvlist_NoticeLesson)
        except Exception:
            print("授業のお知らせの指定のidが存在しない。")

        #授業の情報
        # try:
        #     for elem_tr_CallInfo in driver.find_elements(By.XPATH,'//*[@id="callInfoPublicListForm:_id2:tbody_element"]/tr')[0::2]:
        #         csvlist_CallInfo = []
        #         csvlist_CallInfo.append(elem_tr_CallInfo.text)
        #         writer_NewCallInfo.writerow(csvlist_CallInfo)
        # except:
        #     print("授業の情報の指定のidが存在しない")
            


    def process(self,USERNAME,PASSWORD,token):
        self.USERNAME = USERNAME
        self.PASSWORD = PASSWORD
        self.token = token

        ##個人呼び出し処理    
        #個人呼び出し.csvのfileが存在しない場合は新規作成
        if (os.path.exists(f"{self.USERNAME}個人呼び出し.csv")) == False :
            with open("{}個人呼び出し.csv".format(self.USERNAME),"w",encoding="cp932") as f1:
                writer_CallInfo = csv.writer(f1,lineterminator="\n")
                writer_CallInfo.writerow({})
                
        #個人呼び出しの更新確認    
        CallInfo_chat = []
        with open("{}個人呼び出しnew.csv".format(self.USERNAME)) as CallInfo_new_com, open("{}個人呼び出し.csv".format(self.USERNAME)) as CallInfo_old_com:
            CallInfo_new_com_contents = CallInfo_new_com.readlines()
            CallInfo_old_com_contents = CallInfo_old_com.readlines()
            for line in CallInfo_new_com_contents:
                if line not in CallInfo_old_com_contents:
                    CallInfo_chat.append(line)
                    
        #コピー
        shutil.copy('{}個人呼び出しnew.csv'.format(self.USERNAME),'{}個人呼び出し.csv'.format(self.USERNAME))
        CallInfo_new_com.close()


        ##授業のお知らせ処理
        #授業のお知らせ.csvのfileが存在しない場合は新規作成
        if (os.path.exists(f"{self.USERNAME}授業のお知らせ.csv")) == False :
            with open("{}授業のお知らせ.csv".format(self.USERNAME),"w",encoding="cp932") as f2:
                writer_CallInfo = csv.writer(f2,lineterminator="\n")
                writer_CallInfo.writerow({})
        #授業のお知らせの更新確認    
        NoticeLesson_chat = []
        with open("{}授業のお知らせnew.csv".format(self.USERNAME)) as NoticeLesson_new_com, open("{}授業のお知らせ.csv".format(self.USERNAME)) as NoticeLesson_old_com:
            NoticeLesson_new_com_contents = NoticeLesson_new_com.readlines()
            NoticeLesson_old_com_contents = NoticeLesson_old_com.readlines()
            for line in NoticeLesson_new_com_contents:
                if line not in NoticeLesson_old_com_contents:
                    NoticeLesson_chat.append(line)
        #コピー
        shutil.copy('{}授業のお知らせnew.csv'.format(self.USERNAME),'{}授業のお知らせ.csv'.format(self.USERNAME))
        NoticeLesson_new_com.close()


        
        ##lineに通知する内容を決める
        if CallInfo_chat == [] and NoticeLesson_chat == []:
            print("終わり")

        elif CallInfo_chat == []:
            chat = "\n授業のお知らせ\n"
            for m in NoticeLesson_chat:
                chat += m
            
        elif NoticeLesson_chat == []:
            chat = "\n個人呼び出し\n"
            for i in CallInfo_chat:
                chat += i
                
        else:
            chat =  "\n個人呼び出し\n"
            for i in CallInfo_chat:
                chat += i
            chat += "\n授業のお知らせ\n"
            for m in NoticeLesson_chat:
                chat += m


        with open(f"{self.USERNAME}.txt","w",encoding="cp932") as f4:
            f4.writelines(chat)
        
        # lineからtokenを取得
        try:
            
            url="https://notify-api.line.me/api/notify"
            auth={"Authorization":"Bearer "+self.token}

            content={"message":chat}
            requests.post(url, headers=auth, data=content)
        except:#chat[]の時
            print("respond[400]更新してない")

        

In [35]:
USERNAME = "0215615"
PASSWORD = "yoshthko1"
token = "A6aoyO8Fxaz7MTLT3nO7PnHnlBwyO9f6dwxgQIPeoh4"
yoshi = StudentPortalSite_NoticeLine(USERNAME,PASSWORD)
yoshi.process(USERNAME,PASSWORD,token)